# RAG Exercise Solution

In [1]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nuntrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [3]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrtition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [4]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrition advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrtition_qna_tool],
)

In [5]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are guidance notes and a sample approach:

- Best meal choices:
  - Iron-rich foods: lean red meat, poultry, fish, eggs, beans, lentils, fortified cereals; pair with vitamin C-rich foods (citrus, peppers) to improve absorption.
  - Folate: leafy greens, legumes, fortified grains.
  - Calcium and vitamin D: dairy or fortified dairy alternatives, yogurt, cheese; plus fortified plant milks.
  - Protein: lean meats, fish (low-mercury), eggs, dairy, tofu, tempeh, legumes.
  - Whole grains, fruits, vegetables, nuts, seeds for fiber and micronutrients.
  - Omega-3 DHA: fatty fish (salmon, sardines) a few times a week or a supplement if advised.
- Calorie guidance: an extra ~200 calories per day is a common baseline; total needs vary by trimester and activity. Some guidelines suggest ~300–500 extra kcal in later pregnancy, but many reliable sources note about 200 kcal/day as a practical add-on.

Sample day (roughly balanced; adjust portions to meet your needs):
- Breakfast: fortified cere